In [ ]:
import pennylane as qml
from pennylane import numpy as np

from tqdm import tqdm
from IPython.display import clear_output

import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from functools import reduce
import random
import math
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [ ]:
# 기본 세팅
name1 = "AAPL"
name2 = "TSLA"
cut_left = -0.1
cut_right = 0.1
num_bins = 16
num_dataset = 2000

## 금융 데이터를 yfinance로부터 받아오기

In [ ]:
import yfinance as yf
import datetime

def get_stocks_datas(ticker_names: list[str], start_date, end_date):
    # WARN: returns는 prices보다 길이가 1 작다.
    prices = {}
    returns = {}
    for name in ticker_names:
        data = yf.Ticker(name)
        data = data.history(start=start_date, end=end_date, interval="1d")
        prices[name] = data['Close']
        returns[name] = data['Close'].pct_change()[1:]
    return prices, returns

start_date = datetime.datetime(2011, 1, 1)
end_date = datetime.datetime(2022, 12, 31)
stock_prices, stock_returns = get_stocks_datas([name1, name2], start_date, end_date)


In [ ]:
import numpy as np

# 분포의 특성 설정
mean1 = 0.03
mean2 = -0.03
std_dev = 0.025
correlation = 0.4

# 공분산 계산
cov_matrix = np.array([
    [std_dev ** 2, correlation * std_dev ** 2],
    [correlation * std_dev ** 2, std_dev ** 2]
])

# 다변수 정규분포 샘플 생성
num_samples = 2000
samples = np.random.multivariate_normal([mean1, mean2], cov_matrix, num_samples)

# 생성된 샘플 확인
print("Generated samples:\n", samples)
samples.shape

In [ ]:
stock_returns[name1] = samples[:, 0]
stock_returns[name2] = samples[:, 1]

In [ ]:
# 산점도 그리기
plt.scatter(stock_returns[name1], stock_returns[name2], alpha=0.5)
plt.title(f'{name1}/{name2} daily returns')
plt.xlabel(name1)
plt.ylabel(name2)
plt.xlim(-0.2, 0.2)
plt.ylim(-0.3, 0.3)
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bin_edges = np.linspace(cut_left, cut_right, num_bins+1)

plt.figure(figsize=(10, 6))

plt.hist(stock_returns[name1], bins=bin_edges, alpha=0.7, label=name1)
plt.hist(stock_returns[name2], bins=bin_edges, alpha=0.7, label=name2)

plt.title(f'Distribution of Daily Returns: {name1} vs. {name2}')
plt.xlabel('Daily Return')
plt.ylabel('Frequency')
plt.legend()
plt.show()

print(f"Mean of {name1}: {stock_returns[name1].mean():.4f}")
print(f"Mean of {name2}: {stock_returns[name2].mean():.4f}")


## risk-return space에 그려보자
앞으로 우리가 뽑을 최종 결과물이다

In [ ]:
x = []
y = []
c = []
for k in np.linspace(0, 1, 100):
    merged_returns = k * stock_returns[name1] + (1 - k) * stock_returns[name2]
    x.append(merged_returns.std())
    y.append(merged_returns.mean())
    c.append(k)

plt.scatter(x, y, c=c, cmap='viridis')
plt.xlabel('Standard Deviation')
plt.ylabel('Mean')
plt.title(f'std-mean plot {name1}/{name2}')
plt.colorbar()
plt.show()

## Dataset 생성

linespace로 균일하게 내분해서 절반, uniform으로 절반 생성한다

In [ ]:
dataset = []
def add_dataset(alpha):
    # 첫번째 주식을 alpha, 두번째 주식을 1-alpha만큼.
    bins = np.linspace(-0.1, 0.1, num_bins + 1)
    returns = alpha * stock_returns[name1] + (1 - alpha) * stock_returns[name2]
    hist, _ = np.histogram(returns, bins=bins)
    dataset.append(hist/sum(hist))

for t in np.linspace(0, 1, num_dataset//2): # 절반은 직접 내분
    add_dataset(t)
for t in np.random.uniform(0, 1, num_dataset//2): # 절반은 랜덤
    add_dataset(t)

dataset = np.array(dataset)
print(dataset.shape)

In [ ]:
# setting torch device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
noise_qubits = 4
code_qubits = 1
output_qubits = 4 # 실제로 출력 상태를 만드는데 쓰이는 개수
n_qubits = noise_qubits
assert(output_qubits <= n_qubits)

n_layers = 10
BATCH_SIZE = 16

print("n_qubits = {} n_layers = {}".format(n_qubits, n_layers))
dev = qml.device("default.qubit", wires=n_qubits) # 제너레이터 돌려서 값 뽑아내는데 쓰임. 이중 몇개 골라서 판별자도 학습함.

In [ ]:
coeff = 0.05

def generator_init(generator_input):
    for i in range(n_qubits):
        qml.RY((generator_input[i]-0.5) * np.pi/4, wires=i) # TODO: *a 해서 값 범위 맞추기

def generator_layer(params):
    for i in range(n_qubits):
        qml.RY(params[i][0], wires=i)
    for i in range(n_qubits):
        qml.CZ(wires=[i, (i+1)%n_qubits])

@qml.qnode(dev, interface="torch")
def generator_circuit(params, generator_input):
    generator_init(generator_input)
    for param in params:
        generator_layer(param)
    return qml.probs(wires=range(output_qubits))


def generator_forward(params, generator_input):
    generator_output = [generator_circuit(params, single_in) for single_in in generator_input] # (BATCH_SIZE, 2**output_qubits) 차원
    generator_output = torch.stack(generator_output) # (BATCH_SIZE, 2**output_qubits) 차원
    return generator_output


def generator_train_step(params, generator_input, use_mine = False, _qmine = False):
    code_input = generator_input[:, -code_qubits:]

    generator_output = generator_forward(params, generator_input)
    generator_output = generator_output.to(torch.float32) # (BATCH_SIZE, output_qubits)
    
    disc_output = discriminator(generator_output) # 밑에 코드에서 정의됨
    gan_loss = -torch.log(disc_output).mean()
    

    if use_mine:
        pred_xy = mine(code_input, generator_output)
        code_input_shuffle = code_input[torch.randperm(BATCH_SIZE)]
        pred_x_y = mine(code_input_shuffle, generator_output)
        mi = torch.mean(pred_xy) - torch.log(torch.mean(torch.exp(pred_x_y)))
        gan_loss -= coeff * mi


    return generator_output, gan_loss

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class LinearDiscriminator(nn.Module):
    def __init__(self, input_dim=num_bins):
        super().__init__()
        self.hidden_size = 100
        self.input_dim = input_dim
        self.layers = nn.Sequential(
            nn.Linear(num_bins, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, 1),
            nn.Sigmoid())


    def forward(self, x):
        return self.layers(x)


class LinearMine(nn.Module):
    def __init__(self):
        super(LinearMine, self).__init__()
        H = 200
        self.fc1 = nn.Linear(code_qubits, H)
        self.fc2 = nn.Linear(2**output_qubits, H)
        self.fc3 = nn.Linear(H, 1)

    def forward(self, x, y):
        h1 = F.relu(self.fc1(x)+self.fc2(y))
        h2 = self.fc3(h1)
        return h2


disc_loss_fn = nn.BCELoss()
def disc_cost_fn(real_input, fake_input, smoothing=False):
    batch_num = real_input.shape[0]

    disc_real = discriminator(real_input)
    disc_fake = discriminator(fake_input)

    real_label = torch.ones((batch_num, 1)).to(device)
    fake_label = torch.zeros((batch_num, 1)).to(device)
    
    if smoothing:
        real_label = real_label - 0.2*torch.rand(real_label.shape).to(device)
    
    loss = 0.5 * (disc_loss_fn(disc_real, real_label) + disc_loss_fn(disc_fake, fake_label))
    
    return loss

In [ ]:
generator_params = Variable(torch.tensor(np.random.normal(-np.pi, np.pi, (n_layers, n_qubits, 1))), requires_grad=True)
print("parameter shape: ", generator_params.shape)
discriminator = LinearDiscriminator()
mine = LinearMine()

G_lr = 4e-4
D_lr = 4e-5
M_lr = 1e-3
use_mine = True
use_qmine = False
G_opt = torch.optim.Adam([generator_params], lr=G_lr)
D_opt = torch.optim.Adam(discriminator.parameters(), lr=D_lr)
M_opt = torch.optim.Adam(mine.parameters(), lr=M_lr)

G_scheduler = torch.optim.lr_scheduler.StepLR(G_opt, step_size=30, gamma=0.7)
D_scheduler = torch.optim.lr_scheduler.StepLR(D_opt, step_size=30, gamma=0.85)
M_scheduler = torch.optim.lr_scheduler.StepLR(M_opt, step_size=30, gamma=0.7)

In [ ]:
import os
############# 가장 최신 버전 폴더로 저장한다 #############
file_num = 1
title = f'mine={use_mine}_try={file_num}'
if not os.path.exists(f'result/{title}'):
    os.makedirs(f'result/{title}')
else:
    while os.path.exists(f'result/{title}'):
        file_num += 1
        title = f'mine={use_mine}_try={file_num}'
        if not os.path.exists(f'result/{title}'):
            os.makedirs(f'result/{title}')
            break
###################################################

with open(f'result/{title}/param.txt', 'w') as f:
    f.write('사용주식1 = {} 사용주식2 = {}\n'.format(name1,name2))
    f.write('G_lr = {}\n'.format(G_lr))
    f.write('D_lr = {}\n'.format(D_lr))
    f.write('M_lr = {}\n'.format(M_lr))
    f.write('G_scheduler: step={}, gamma={}\n'.format(G_scheduler.step_size, G_scheduler.gamma))
    f.write('D_scheduler: step={}, gamma={}\n'.format(D_scheduler.step_size, D_scheduler.gamma))
    f.write('M_scheduler: step={}, gamma={}\n'.format(M_scheduler.step_size, M_scheduler.gamma))
    f.write('coeff = {}\n'.format(coeff))
    f.write('use_mine = {}\n'.format(use_mine))
    f.write('use_qmine = {}\n'.format(use_qmine))
    f.write('n_qubits = {}\n'.format(n_qubits))
    f.write('noise_qubits = {}\n'.format(noise_qubits))
    f.write('code_qubits = {}\n'.format(code_qubits))
    f.write('n_layers = {}\n'.format(n_layers))
    f.write('param shape = {}\n'.format(generator_params.shape))

In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt

def visualize_output(gen_outputs, gen_codes, means, stds, title, rep, recorder):
    warnings.simplefilter("ignore")


    random_indices = np.random.choice(gen_outputs.shape[0], size=9, replace=False)
    selected_samples = [gen_outputs[i] for i in random_indices]
    selected_codes = [gen_codes[i][0] for i in random_indices]

    sorted_indices = np.argsort(selected_codes)
    sorted_selected_codes = [selected_codes[i] for i in sorted_indices]
    sorted_selected_samples = [selected_samples[i] for i in sorted_indices]

    plt.figure(figsize=(12, 8))

    for i, sample in enumerate(sorted_selected_samples):
        plt.subplot(3, 3, i + 1)
        plt.bar(np.linspace(-0.1, 0.1, num_bins), sample, width=0.2*(num_bins+1)/num_bins**2)
        plt.title(f'{sorted_selected_codes[i]:.3f}')
        plt.ylim(0, 0.3)
        plt.xlabel('Return')
        plt.ylabel('Frequency')

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.suptitle(f"Epoch {rep}, random imgs")
    plt.savefig(f'result/{title}/samples_{rep}.png')
    if rep % 5 == 0:
        plt.show()
    plt.clf()
    plt.close()
    
    plt.figure(figsize=(10, 4))
    plt.title('Epoch {0} recorder'.format(rep))
    plt.plot(recorder['mi'], label='mi', marker='o')
    plt.plot(recorder['d_loss'], label='d_loss', marker='o')
    plt.plot(recorder['g_loss'], label='g_loss', marker='o')

    for i in range(code_qubits):
        plt.plot(recorder[f'c{i}-mean'], label=f'c{i}-mean', marker='o')
        plt.plot(recorder[f'c{i}-std'], label=f'c{i}-std', marker='o')

    plt.xlabel('epoch')
    plt.ylabel('values')
    plt.ylim(-1, 1)
    plt.title(f'record (rep={rep})')
    plt.legend()
    plt.savefig(f'result/{title}/record_{rep}.png', dpi=100)
    if rep % 5 == 0:
        plt.show()
    plt.clf()
    plt.close()

    plt.figure(figsize=(6 + 4 * code_qubits, 4))  # 전체 그림의 크기 지정
    for i in range(code_qubits):
        plt.subplot(1, code_qubits, i+1)
        plt.title('Epoch {0} code {1}'.format(rep, i))
        plt.scatter(stds, means, s=15, c=gen_codes[:, i], cmap='RdYlBu', alpha=0.6)
        plt.colorbar()  # 색상 막대 추가
        plt.grid()
    
    plt.savefig(f'result/{title}/mean_std_{rep}.png', dpi=100)
    if rep % 5 == 0:
        plt.show()
    plt.clf()
    plt.close()

def calculate_mean_std(probs):
    d = (cut_right - cut_left) / num_bins * (num_bins // 2 - 0.5) # 히스토그램 오른쪽 끝 중심값
    vals = np.linspace(-d, d, num_bins)
    mean = np.sum(probs * vals) # 평균 계산
    std = np.sqrt(np.sum(probs * (vals - mean)**2)) # 표준편차 계산
    return mean, std

def calculate_corr(gen_outputs, code_inputs):
    # gen_outputs: (B, bin_num), code_inputs: (B, code_qubits)
    means = []
    stds = []
    for i in range(len(gen_outputs)):
        mean, std = calculate_mean_std(gen_outputs[i])
        means.append(mean)
        stds.append(std)
    mean_corrs = []
    std_corrs = []
    for i in range(code_qubits):
        mean_corrs.append(np.corrcoef(means, code_inputs[:,i])[0,1])
        std_corrs.append(np.corrcoef(stds, code_inputs[:,i])[0,1])
    
    return mean_corrs, std_corrs, means, stds

In [ ]:
import openpyxl
epoch = 1500

recorder_keywords = ['d_loss', 'g_loss', 'mi']
for i in range(code_qubits):
    recorder_keywords.append(f'c{i}-mean')
    recorder_keywords.append(f'c{i}-std')
recorder = {k: [] for k in recorder_keywords}
final_rep = 0

for rep in range(1, epoch+1):
    np.random.shuffle(dataset)
    iter_num = int(len(dataset) * 0.25 //BATCH_SIZE) # 매번 50% 추출해서 학습. 셔플하니까 자투리 생기는건 무시.

    G_loss_sum = 0.0
    D_loss_sum = 0.0
    mi_sum = 0.0

    log_gen_outputs = []
    log_gen_codes = []
    pbar = tqdm(range(iter_num))

    for i in pbar:
        batch = torch.FloatTensor(dataset[BATCH_SIZE * i : BATCH_SIZE * i + BATCH_SIZE])

        # train generator
        generator_seed = torch.rand((BATCH_SIZE, n_qubits))
        generator_output, generator_loss = generator_train_step(generator_params, generator_seed, use_mine=use_mine)
        G_opt.zero_grad()
        generator_loss.requires_grad_(True)
        generator_loss.backward()
        G_opt.step()

        # train discriminator
        fake_input = generator_output.detach().to(torch.float32)
        disc_loss = disc_cost_fn(batch, fake_input, smoothing=False)
        D_opt.zero_grad()
        disc_loss.requires_grad_(True)
        disc_loss.backward()
        D_opt.step()

        # train mine
        code_input = generator_seed[:, -code_qubits:] 
        pred_xy = mine(code_input, fake_input)
        code_input_shuffle = code_input[torch.randperm(BATCH_SIZE)]
        pred_x_y = mine(code_input_shuffle, fake_input)
        mi = -torch.mean(pred_xy) + torch.log(torch.mean(torch.exp(pred_x_y)))
        M_opt.zero_grad()
        mi.requires_grad_(True)
        mi.backward()
        M_opt.step()

        D_loss_sum += disc_loss.item()
        G_loss_sum += generator_loss.item()
        mi_sum -= mi.item() # (-1)곱해져 있어서 빼야함.

        pbar.set_postfix({'G_loss': G_loss_sum/(i+1), 'D_loss': D_loss_sum/(i+1), 'MI': mi_sum/(i+1)})
        log_gen_outputs.append(fake_input.numpy())
        log_gen_codes.append(code_input.numpy())

    G_scheduler.step()
    D_scheduler.step()
    M_scheduler.step()

    recorder['d_loss'].append(D_loss_sum/iter_num)
    recorder['g_loss'].append(G_loss_sum/iter_num)
    recorder['mi'].append(mi_sum/iter_num)

    print("epoch: {}, D_loss: {}, G_loss: {}, MI = {}".format(rep, D_loss_sum/iter_num, G_loss_sum/iter_num, mi_sum/iter_num))
    
    log_gen_outputs = np.concatenate(log_gen_outputs, axis=0)
    log_gen_codes = np.concatenate(log_gen_codes, axis=0)
    
    mean_corrs, std_corrs, means, stds = calculate_corr(log_gen_outputs, log_gen_codes)
    for i in range(code_qubits):
        recorder[f'c{i}-mean'].append(mean_corrs[i])
        recorder[f'c{i}-std'].append(std_corrs[i])

    visualize_output(log_gen_outputs, log_gen_codes, means, stds, title, rep, recorder)
    
    with open(f'result/{title}/Opts_{rep}.pkl', 'wb') as file:
        pickle.dump((G_opt, D_opt, M_opt), file)
    with open(f'result/{title}/schedulers_{rep}.pkl', 'wb') as file:
        pickle.dump((G_scheduler, D_scheduler, M_scheduler), file)
    with open(f'result/{title}/discriminator_{rep}.pkl', 'wb') as file:
        pickle.dump(discriminator, file)
    with open(f'result/{title}/generator_{rep}.pkl', 'wb') as file:
        pickle.dump(generator_params, file)

    df = pd.DataFrame(recorder)
    output_filename = f'result/{title}/recorder.xlsx'
    df.to_excel(output_filename, index=False)

    final_rep = rep

## 최종 결과 plot

In [ ]:
inputs = []
outputs = []

for i in tqdm(range(5000)):
    with torch.no_grad():
        z = np.random.uniform(-1, 1, (1, n_qubits, 1))
        code_input = z[:, -code_qubits:].reshape(code_qubits) # 입력 z중에서 code를 추출한다.
        generator_output = generator_forward(generator_params, z, copula=output_copula)
        generator_output = generator_output.cpu().numpy().reshape(2)
        outputs.append(generator_output)
        inputs.append(code_input)

inputs = np.array(inputs).reshape(-1, code_qubits)

for code_ind in range(code_qubits):
    outputs = np.array(outputs)
    plt.scatter(outputs[:, 0], outputs[:, 1], c=inputs[:, code_ind], cmap='RdYlBu', alpha=0.2)
    plt.colorbar()  # 색상 막대 추가
    plt.title(f'code{code_ind}-distribution (rep = {final_rep})')
    plt.savefig(f'result/{title}/code_{code_ind}_{final_rep}.png')
    plt.show()